# Train Networks

- Train SoftMax or Multi-BCE classifier for the EEG diagnosis classification
    - CAUEEG-task1 benchmark: Classification of **Normal** and **Abnormal** symptoms
    - CAUEEG-task2 benchmark: Classification of **Normal**, **MCI**, and **Dementia** symptoms
- `Weights and Biases` sweep is used for hyperparameter search

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ..

C:\Users\Minjae\Desktop\EEG_Project


In [2]:
# Load some packages
import os
import json
from copy import deepcopy

import hydra
from omegaconf import OmegaConf
import yaml
import wandb
import pprint

# custom package
from run_train import check_device_env
from run_train import prepare_and_run_train

## Environment

In [3]:
data_cfg_file = 'task2'
train_cfg_file = 'tiny_train'
device = 'cuda:0'
sweep_yaml_path = 'config/sweep/sweep_task2.yaml'
count = 1

## Sweep configurations

In [4]:
with open(sweep_yaml_path) as f:
    sweep_yaml = yaml.load(f, Loader=yaml.FullLoader)

sweep_yaml.pop('command')
pprint.pprint(sweep_yaml)

{'entity': 'ipis-mjkim',
 'method': 'random',
 'name': 'sweep-task2',
 'parameters': {'data.EKG': {'values': ['O', 'X']},
                'data.awgn': {'distribution': 'uniform', 'max': 0.12, 'min': 0},
                'data.awgn_age': {'distribution': 'uniform',
                                  'max': 0.3,
                                  'min': 0},
                'data.mgn': {'distribution': 'uniform', 'max': 0.1, 'min': 0},
                'data.photic': {'values': ['O', 'X']},
                'data.seq_length': {'values': [1000, 2000, 3000, 4000]},
                'model': {'values': ['1D-VGG-16',
                                     '1D-VGG-19',
                                     '1D-ResNet-18',
                                     '1D-ResNet-50',
                                     '1D-ResNeXt-50',
                                     '1D-ResNeXt-101',
                                     '1D-Wide-ResNet-50',
                                     '1D-CNN-Transformer',
      

In [5]:
sweep_id = wandb.sweep(sweep_yaml, project=sweep_yaml['project'])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 3aektdjh
Sweep URL: https://wandb.ai/ipis-mjkim/sweep-test/sweeps/3aektdjh


## Train

In [6]:
def train_sweep():
    # connect wandb
    wandb_run = wandb.init()
    wandb.run.name = wandb.run.id
    
    with wandb_run:
        # init hydra
        with hydra.initialize(config_path='../config'):
            add_configs = [f"data={data_cfg_file}", 
                           f"model={wandb.config.model}",
                           f"train={train_cfg_file}",
                           f"++train.device={device}",]

            cfg = hydra.compose(config_name='default', overrides=add_configs)

            config = {**OmegaConf.to_container(cfg.data), 
                      **OmegaConf.to_container(cfg.train),
                      **OmegaConf.to_container(cfg.model)}

            check_device_env(config)

        # load default configurations not selected by wandb.sweep
        cfg_sweep = dict()
        for k, v in config.items():
            if k not in [wandb_key.split('.')[-1] for wandb_key in wandb.config.keys()]:
                cfg_sweep[k] = v

        # load the selected configurations from wandb sweep with preventing callables from type-conversion to str
        for k, v in wandb.config.items():
            k = k.split('.')[-1]
            if k not in cfg_sweep:
                cfg_sweep[k] = v

        config = cfg_sweep
        config['wandb_run'] = wandb_run
        pprint.pprint(config)

        # build the dataset and train the model
        if config.get('ddp', False):
            mp.spawn(prepare_and_run_train,
                     args=(config['ddp_size'], config,),
                     nprocs=config['ddp_size'],
                     join=True)
        else:
            prepare_and_run_train(rank=None, world_size=None, config=config)

In [7]:
wandb.agent(sweep_id, function=train_sweep, count=count)

wandb: Agent Starting Run: 0uv4ashm with config:
wandb: 	data.EKG: X
wandb: 	data.awgn: 0.0581451723266512
wandb: 	data.awgn_age: 0.2243544676293716
wandb: 	data.mgn: 0.027709291109425185
wandb: 	data.photic: X
wandb: 	data.seq_length: 1000
wandb: 	model: 2D-VGG-19
wandb: 	model.activation: mish
wandb: 	model.dropout: 0.13331455206298637
wandb: 	model.fc_stages: 5
wandb: 	model.use_age: fc
wandb: 	train.criterion: cross-entropy
wandb: 	train.lr_scheduler_type: constant_with_decay
wandb: 	train.mixup: 0
wandb: 	train.num_history: 2
wandb: 	train.search_lr: False
wandb: 	train.total_samples: 500
wandb: 	train.weight_decay: 0.01505068238940257
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ipis-mjkim. Use `wandb login --relogin` to force relogin


{'EKG': 'X',
 '_target_': 'models.vgg_2d.VGG2D',
 'activation': 'mish',
 'awgn': 0.0581451723266512,
 'awgn_age': 0.2243544676293716,
 'base_lr': 0.0001,
 'batch_norm': True,
 'criterion': 'cross-entropy',
 'crop_multiple': 4,
 'crop_timing_analysis': False,
 'dataset_path': 'local/dataset/02_Curated_Data_220419/',
 'ddp': False,
 'device': device(type='cuda', index=0),
 'draw_result': True,
 'dropout': 0.13331455206298637,
 'fc_stages': 5,
 'file_format': 'memmap',
 'in_channels': '???',
 'input_norm': 'dataset',
 'latency': 2000,
 'load_event': False,
 'lr_scheduler_type': 'constant_with_decay',
 'mgn': 0.027709291109425185,
 'minibatch': 160,
 'mixup': 0,
 'model': '2D-VGG-19',
 'num_history': 2,
 'out_dims': '???',
 'photic': 'X',
 'run_mode': 'train',
 'save_model': False,
 'search_lr': False,
 'search_multiplier': 1.0,
 'seq_len_2d': '???',
 'seq_length': 1000,
 'signal_length_limit': 10000000,
 'task': 'task2',
 'test_crop_multiple': 8,
 'total_samples': 500,
 'use_age': 'fc',
 

wandb: WARNING Config item 'model' was locked by 'sweep' (ignored update).


Learning Rate,█▁▁
Loss,█▄▁
Multi-Crop Test Accuracy,▁
Test Accuracy,▁
Train Accuracy,▁▁█
Validation Accuracy,▁▁▁
Learning Rate,1e-05
Loss,1.0921
Multi-Crop Test Accuracy,50.0
Test Accuracy,38.98305
Train Accuracy,45.625
